# cuSpatial API demo
GTC April 2023 Michael Wang and Thomson Comer


## Data

[National Address Database](https://nationaladdressdata.s3.amazonaws.com/NAD_r12_TXT.zip)

[NYC Taxi Boroughs](https://d37ci6vzurychx.cloudfront.net/misc/taxi_zones.zip)

[taxi2015.csv](https://rapidsai-data.s3.us-east-2.amazonaws.com/viz-data/nyc_taxi.tar.gz)

The following notebook demonstrates the use of cuSpatial to perform analytics using large datasets.

The structure of the notebook is as follows:
1. Imports
1. Read datasets: National Address Database (NAD), NYC Taxi Boroughs Polygons, 2015 NYC Taxi pickup/dropoff information with lon/lat. Also convert epsg:2263 (NYC Long Island) to WGS.
1. Convert separate lon/lat columns in DataFrames into cuspatial.GeoSeries
1. Compute number of addresses and pickups in each borough
1. Compute addresses for each pickup in one borough

A drawing of an addresses table and a pickups table, with a line connecting two rows together and
adding the address where it belongs in the pickups table.

In [1]:
import cudf
import cuspatial
import geopandas
import cupy as cp
import pandas as pd
cudf.set_option("spill", True) 

<p align="center">
    <img src="https://www.transportation.gov/sites/dot.gov/files/images/NAD_Partners_20221201_v12_Release_0.jpg" width="350" height="400">
    <img src="https://www.dropbox.com/s/b7zmjlxnrtgqdwn/zones.png?dl=1" width="350" height="400">
    <img src="https://www.dropbox.com/s/i516rqruz97fd2q/nyc-taxi-pickups.png?dl=1", width="430">
</p>

I/O
 
- National Address Database (NAD): 
- NYC Taxi Zones Shapefile (zones)
- NYC 2015 Taxi Pickups and Dropoffs with Lon/Lat Coords (taxi2015)

In [2]:
# I/O (18GB NAD, 265 borough polygons, 13m taxi pickups and dropoffs.
NAD = cudf.read_csv('NAD_r11.txt', usecols=[
    'State',
    'Longitude',
    'Latitude',
])
NAD = NAD[NAD['State'] == 'NY']
NAD_Street = cudf.read_csv('NAD_r11.txt', usecols=[
    'State',
    'StN_PreDir',
    'StreetName',
    'StN_PosTyp',
    'Add_Number',
])
NAD_Street = NAD_Street[NAD_Street['State'] == 'NY']
# Read taxi_zones.zip shapefile with GeoPandas, then convert to epsg:4326 for lon/lat
host_zones = geopandas.read_file('taxi_zones.zip')
host_lonlat = host_zones.to_crs(epsg=4326)
zones = cuspatial.from_geopandas(host_lonlat)
zones.set_index(zones['OBJECTID'], inplace=True)
taxi2015 = cudf.read_csv('taxi2015.csv')

`make_geoseries_from_lon_lat`
<center><img src="https://www.dropbox.com/s/pp75u59z5uxwrlz/table-to-geoseries.png?dl=1" width=500></center>

In [55]:
# Utility function to convert dataframes into GeoSeries

def make_geoseries_from_lon_lat(lon, lat):
    # Scatter the two columns into one column
    assert len(lon) == len(lat)
    xy = cudf.Series(cp.zeros(len(lon) * 2), dtype="float32")
    xy[::2] = lon
    xy[1::2] = lat

    return cuspatial.GeoSeries(cuspatial.core._column.geocolumn.GeoColumn._from_points_xy(xy._column))

In [56]:
# Convert DataFrames to GeoSeries

pickups = make_geoseries_from_lon_lat(
    taxi2015['pickup_longitude'],
    taxi2015['pickup_latitude']
)
addresses = make_geoseries_from_lon_lat(
    NAD['Longitude'],
    NAD['Latitude']
)

In [73]:
taxi2015['geometry'] = pickups
taxi = cuspatial.GeoDataFrame(taxi2015)
cuspatial.GeoSeries(cuspatial.core._column.geocolumn.GeoColumn._from_points_xy(
    cudf.core.column.column.as_column(cp.arange(10, dtype=cp.float32))
))._column._meta.union_offsets
pickups._column._meta.union_offsets
taxi['geometry']._column._meta.union_offsets

from shapely.geometry import *
test = cuspatial.GeoDataFrame(cudf.DataFrame({
    'a': [0, 1], 'geometry': cuspatial.GeoSeries([Point(0, 0), Point(1, 1)])
}))
import pyarrow as pa
pa.parquet.write_table(test.to_arrow(), 'table.parquet')


ArrowNotImplementedError: Unhandled type for Arrow to Parquet schema conversion: dense_union<0: list<item: double>=0, 1: list<item: null>=1, 2: list<item: null>=2, 3: list<item: null>=3>

In [5]:
borough_addresses = zones['geometry'].contains_properly(addresses, allpairs=True)
display(borough_addresses)

,polygon_index,point_index
17872,1,5648100
17873,1,5648101
17874,2,5202801
17875,2,5202802
17876,2,5202803
...,...,...
966784,262,5368821
966785,262,5368822
966786,262,5368823
966787,262,5368824


In [6]:
borough_pickups = zones['geometry'].iloc[0:120].contains_properly(pickups, allpairs=True)
display(borough_pickups)

# You can do it one of two ways: .contains_properly, or write the pip yourself.

,polygon_index,point_index
17552,0,44084
17553,0,76169
17554,0,129737
17555,0,177939
17556,0,219859
...,...,...
3408220,119,12253904
3408221,119,12574064
3408222,119,12634955
3408223,119,12666699


In [7]:
# Add pickup and address counts to zones dataframe

zones["pickup_count"] = borough_pickups.groupby('polygon_index').count()
zones["address_count"] = borough_addresses.groupby('polygon_index').count()
pd.set_option('display.max_rows', 265)
zones.iloc[0:120]

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry,pickup_count,address_count
OBJECTID,,,,,,,,,
1,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((-74.18445 40.69500, -74.18449 40.695...",18,2
2,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ...",30,5802
3,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((-73.84793 40.87134, -73.84725 40.870...",44433,810
4,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((-73.97177 40.72582, -73.97179 40.725...",3,7953
5,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((-74.17422 40.56257, -74.17349 40.562...",18,4153
6,6,0.150491,0.000606,Arrochar/Fort Wadsworth,6,Staten Island,"POLYGON ((-74.06367 40.60220, -74.06351 40.602...",29723,8283
7,7,0.107417,0.000390,Astoria,7,Queens,"POLYGON ((-73.90414 40.76752, -73.90325 40.767...",183,<NA>
8,8,0.027591,0.000027,Astoria Park,8,Queens,"POLYGON ((-73.92334 40.77513, -73.92398 40.774...",64,5755
9,9,0.099784,0.000338,Auburndale,9,Queens,"POLYGON ((-73.78502 40.76104, -73.78486 40.760...",1324,7860


# Computing distances
## Cartesian product via tiling

<center><img src="https://www.dropbox.com/s/wlcr9fugq79nyut/tiled-cartesian-product.png?dl=1" width=650></center>

In [8]:
BOROUGH_ID = 12

# Let's make two GeoSeries: For each borough, create a GeoSeries with all address Points
# repeated the number of times there are pickups in that borough, and another GeoSeries with
# the opposite: all pickups Points repeated the number of times there are addresses in that
# borough.

# addresses tiled
borough_address_point_ids = borough_addresses['point_index'][borough_addresses['polygon_index'] == BOROUGH_ID]
pickups_count = len(borough_pickups[borough_pickups['polygon_index'] == BOROUGH_ID])
addresses_tiled = NAD.iloc[
    borough_address_point_ids
].tile(pickups_count)

# pickups tiled
borough_pickup_point_ids = borough_pickups['point_index'][borough_pickups['polygon_index'] == BOROUGH_ID]
addresses_count = len(borough_addresses[borough_addresses['polygon_index'] == BOROUGH_ID])
pickups_tiled = taxi2015[[
    'pickup_longitude',
    'pickup_latitude'
]].iloc[
    borough_pickup_point_ids
].tile(addresses_count)

pickup_points = make_geoseries_from_lon_lat(
    pickups_tiled['pickup_longitude'],
    pickups_tiled['pickup_latitude']
)
address_points = make_geoseries_from_lon_lat(
    addresses_tiled['Longitude'],
    addresses_tiled['Latitude']
)
len(address_points)

11081124

<center><img src="https://www.dropbox.com/s/30rntm6p67mw96c/pairwise_point_distance.png?dl=1" width=550></center>

In [9]:
# get the list of addresses and their indices that are closest to a pickup point

haversines = cuspatial.haversine_distance(
    pickup_points.points.x,
    pickup_points.points.y,
    address_points.points.x,
    address_points.points.y,
)

gb_df = cudf.DataFrame({
    'address': addresses_tiled.index,
    'pickup': pickups_tiled.index,
    'distance': haversines
})

address_indices_of_nearest = gb_df[['address', 'distance']].groupby('address').idxmin()
pickup_indices_of_nearest = gb_df[['pickup', 'distance']].groupby('pickup').idxmin()
address_nearest_pickups = gb_df.loc[address_indices_of_nearest['distance']]
pickups_nearest_address = gb_df.loc[pickup_indices_of_nearest['distance']]

# We're almost there

### We have the index of the addresses and their pickups

In [10]:
# Original data nearest pickups and addresses

nearest_pickups = taxi2015.iloc[pickups_nearest_address['pickup']]
nearest_addresses_lonlat = NAD.loc[pickups_nearest_address['address']]

In [11]:
# Concatenate address fields

def build_address_string(NAD_Street):
    blanks = cudf.Series([' '] * len(NAD_Street))
    blanks.index = NAD_Street.index
    NAD_Street['StN_PreDir'] = NAD_Street['StN_PreDir'].fillna('')
    NAD_Street['StN_PosTyp'] = NAD_Street['StN_PosTyp'].fillna('')
    street_names = NAD_Street['Add_Number'].astype('str').str.cat(
        blanks
    ).str.cat(
        NAD_Street['StN_PreDir']
    ).str.cat(
        blanks
    ).str.cat(
        NAD_Street['StreetName']
    ).str.cat(
        blanks
    ).str.cat(
        NAD_Street['StN_PosTyp']
    )
    return street_names.str.replace('  ', ' ')

nearest_addresses_street_name = NAD_Street.loc[pickups_nearest_address['address']]
street_names = build_address_string(nearest_addresses_street_name)

# Last Step

In [12]:
# Attach the street names to the original pickups dataframe

# save the taxi2015 index
no_index = nearest_pickups.reset_index()
# set taxi2015 street names and distances based on their iloc positions
no_index['pickup_address'] = street_names.reset_index(drop=True)
no_index['distance'] = pickups_nearest_address['distance'].reset_index(drop=True)
# return the index
taxi_pickups_with_address = no_index.set_index(no_index['index'])
taxi_pickups_with_address.drop('index', inplace=True, axis=1)

display(taxi_pickups_with_address[[
    'VendorID',
    'tpep_pickup_datetime',
    'passenger_count',
    'trip_distance',
    'distance',
    'pickup_longitude',
    'pickup_latitude',
    'fare_amount',
    'tip_amount',
    'pickup_address'
]])
display(taxi_pickups_with_address[[
    'pickup_latitude',
    'pickup_longitude',
    'pickup_address',
    'distance'
]].sort_values('distance'))

,VendorID,tpep_pickup_datetime,passenger_count,trip_distance,distance,pickup_longitude,pickup_latitude,fare_amount,tip_amount,pickup_address
index,,,,,,,,,,
10543706,1,2015-01-12 20:17:45,1,1.40,0.040364,-74.015160,40.715694,7.0,1.70,200 North End Avenue
5101140,2,2015-01-30 07:14:57,1,0.67,0.033024,-74.015953,40.711121,4.5,0.00,395 South End Avenue
4784339,1,2015-01-12 10:38:39,1,5.40,0.018222,-74.016479,40.709808,20.5,0.00,250 South End Avenue
9005468,2,2015-01-01 11:57:05,1,0.85,0.005307,-74.017891,40.706558,5.0,1.00,50 Battery Place
2591339,1,2015-01-14 06:56:29,1,6.60,0.063244,-74.015617,40.710674,22.0,2.23,315 South End Avenue
...,...,...,...,...,...,...,...,...,...,...
3653010,2,2015-01-24 13:32:20,1,4.14,0.044396,-74.015388,40.710823,15.5,3.26,1 World Financial Center
8139845,2,2015-01-19 18:15:57,5,7.74,0.049281,-74.014221,40.716946,23.5,0.00,201 Warren Street
3337404,2,2015-01-09 20:17:07,1,0.84,0.036044,-74.015907,40.711014,6.0,1.62,331 South End Avenue


,pickup_latitude,pickup_longitude,pickup_address,distance
index,,,,
11397025,40.711037,-74.016220,345 South End Avenue,0.000358
6985477,40.711037,-74.016220,345 South End Avenue,0.000358
4518329,40.711395,-74.016113,395 South End Avenue,0.000382
4926511,40.710922,-74.016251,339 South End Avenue,0.000462
2966585,40.714256,-74.014244,200 West Street,0.000545
...,...,...,...,...
870182,40.719074,-74.012634,399 Chambers Street,0.201822
2079048,40.718727,-74.012596,400 Chambers Street,0.202765
10234298,40.704365,-74.017159,30 West Street,0.205122


# Use cuXfilter to display these coordinates

In [13]:
import cuxfilter
from bokeh import palettes
from cuxfilter.layouts import feature_and_double_base

from pyproj import Proj, Transformer

display_pickups = taxi2015.iloc[address_nearest_pickups['pickup']]
display_addresses = NAD.loc[address_nearest_pickups['address']]

combined_pickups_and_addresses = cudf.concat([
    display_pickups[['pickup_longitude', 'pickup_latitude']].rename(
        columns={
            'pickup_longitude': 'Longitude',
            'pickup_latitude': 'Latitude'
        }
    ),
    display_addresses[['Longitude', 'Latitude']]], axis=0
)
combined_pickups_and_addresses['color'] = cp.repeat(cp.array([1, 2]), len(
    combined_pickups_and_addresses
)//2)
# Back to NYC CRS for display
transform_4326_to_3857 = Transformer.from_crs('epsg:4326', 'epsg:3857')
combined_pickups_and_addresses['location_x'], combined_pickups_and_addresses['location_y'] = transform_4326_to_3857.transform(
    combined_pickups_and_addresses['Latitude'].values_host, combined_pickups_and_addresses['Longitude'].values_host
)

In [14]:
cux_df = cuxfilter.DataFrame.from_dataframe(combined_pickups_and_addresses)
chart1 = cuxfilter.charts.scatter(
    title="Matched address pickup pairs",
    x='location_x',
    y='location_y',
    color_palette=["Green", "Red"],
    aggregate_col="color", aggregate_fn="mean",
    unselected_alpha=0.0,
    tile_provider="CartoLight", x_range=(-8239910.23,-8229529.24), y_range=(4968481.34,4983152.92),
)
d = cux_df.dashboard([chart1],  theme=cuxfilter.themes.dark, title= 'NYC TAXI DATASSET')

In [15]:
chart1.view()

Card(sizing_mode='scale_width', title='Matched address p..., width=800)
    [0] HoloViews(DynamicMap, height=400, sizing_mode='stretch_both')